In [148]:
kaggledf = df.drop(columns=['project_id', "blurb", "location", "project_url"])
kaggledf.dtypes


backers_count                 int64
category                     object
converted_pledged_amount      int64
country                      object
created_at                    int64
                             ...   
staff_pick                     bool
usd_pledged                 float64
funded                         bool
subcategory                  object
reward_url                   object
Length: 17, dtype: object

In [149]:

train_purpose = ["pledged","funded"]
test_columns = ["category", "country", "created_at", "currency", "deadline", "fx_rate", "goal", "launched_at", "name", "staff_pick", "location", "subcategory"]

test_columns += train_purpose

#drop colums that are not in test_set + reward URL
kaggledf = kaggledf.drop(columns=[col for col in kaggledf if col not in test_columns])

In [150]:
#calculate seconds from UNIX timestamp
kaggledf["seconds"] = kaggledf["deadline"] - kaggledf["created_at"]

In [151]:
#log transform time columns
time_cols = ["seconds","deadline","created_at","launched_at"]
for col in time_cols:
    kaggledf[col] = np.log(kaggledf[col])

In [152]:
#convert all goals to USD    
kaggledf["goal"] = kaggledf["goal"] * kaggledf["fx_rate"]
if "pledged" in kaggledf.columns:
    kaggledf["pledged"] = kaggledf["pledged"] * kaggledf["fx_rate"]

#delete unnessesary colums

kaggledf = kaggledf.drop(columns=["fx_rate", "currency"])

In [153]:
#categorical to numerical data

dummy_category = pd.get_dummies(kaggledf['category'], prefix = "category_")
#dummy_subcategory = pd.get_dummies(kaggledf['subcategory'], prefix = "subcategory_")
#dummy_country = pd.get_dummies(kaggledf['country'], prefix = "country_")
kaggledf = pd.merge(left=kaggledf, right=dummy_category,left_index=True,right_index=True)
#kaggledf = pd.merge(left=kaggledf, right=dummy_subcategory,left_index=True,right_index=True)
#kaggledf = pd.merge(left=kaggledf, right=dummy_country,left_index=True,right_index=True)


    
kaggledf = kaggledf.drop(columns = ["category","subcategory","country", "name"])

In [154]:
#cange staffpick to numerical
kaggledf["staff_pick"] = kaggledf["staff_pick"].apply(lambda x: 1 if x == True else 0)

#change funded to numerical if in df
if "funded" in kaggledf.columns: 
    kaggledf["funded"] = kaggledf["funded"].apply(lambda x: 1 if x == True else 0)


In [155]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
#categorical ranges for goal and pledged, sentiment vader (numerical & categorical)  
#columns to be removed: url's, location, id,
kaggledf.dropna(inplace=True)



kaggledf

,created_at,deadline,goal,launched_at,pledged,staff_pick,funded,seconds,category__art,category__comics,category__crafts,category__dance,category__design,category__fashion,category__film & video,category__food,category__games,category__journalism,category__music,category__photography,category__publishing,category__technology,category__theater
0,21.005418,21.012097,2800.000000,21.010675,3596.000000,0,1,15.999994,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,21.009141,21.013275,4500.000000,21.010686,4586.000000,0,1,15.522812,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,21.010235,21.014573,5000.000000,21.010692,5217.000000,0,1,15.572134,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,21.010204,21.012052,6500.000000,21.010692,7160.000000,0,1,14.717153,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,21.007363,21.012635,15000.000000,21.010692,15445.000000,0,1,15.764505,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,21.022391,21.028212,7000.000000,21.026300,7595.430000,0,1,15.879122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
99996,21.034376,21.036377,25000.000000,21.034480,27196.710000,0,1,14.821609,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
99997,21.035210,21.040802,5500.000000,21.038491,7226.000000,1,1,15.851484,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
99998,21.039664,21.042922,25000.000000,21.040678,31683.000000,0,1,15.314540,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [156]:
kaggledf.dtypes

created_at               float64
deadline                 float64
goal                     float64
launched_at              float64
pledged                  float64
                          ...   
category__music            uint8
category__photography      uint8
category__publishing       uint8
category__technology       uint8
category__theater          uint8
Length: 23, dtype: object

In [157]:
# Assign data from first six columns to X variable
X = kaggledf.iloc[:, 0:6]

# Assign data from first six columns to y variable
y = kaggledf["funded"]

In [164]:

y.head()

0    1
1    1
2    1
3    1
4    1
Name: funded, dtype: int64

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [169]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10, 10, 10), max_iter=1000)
mlp.fit(X_train, y_train.values.ravel())

MLPClassifier(hidden_layer_sizes=(10, 10, 10, 10, 10), max_iter=1000)

In [170]:
predictions = mlp.predict(X_test)

In [171]:
print(confusion_matrix(y_test,predictions))


[[ 8081   154]
 [  131 11634]]


In [172]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      8235
           1       0.99      0.99      0.99     11765

    accuracy                           0.99     20000
   macro avg       0.99      0.99      0.99     20000
weighted avg       0.99      0.99      0.99     20000

